In [1]:
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import os
import sys
sys.path.insert(1, './agent')
sys.path.insert(1, './price_pred')

import stock_env
from QLAgent import QNAgent
from helper import *

In [12]:
name_file_data = 'AAPL.csv'
name_project = 'AXP_batch'
env_name = 'StockEnv-v0'
num_episodes = 1000
len_obs = 50
len_window = 100
interval = 1000
init_balance = 1000
load_model = False
epoch_to_load = 10000
name_model_weights = 'trading_weights'
overlap = 20
batch_size = 1000
action_list=(-0.30, -0.20, 0, 0.20, 0.30)
interval = 100

In [13]:
path_models, path_imgs, path_stats = check_directories(name_project, len_obs, len_window)
np.random.seed(42)
# Read data you want to use for trading
train_data, test_data = get_data(f'./data/{name_file_data}')

In [14]:
# Create an instant
env = gym.make(env_name, train_data=train_data, eval_data=test_data, 
               len_obs=len_obs, len_window=len_window, init_balance=init_balance,
               action_list=action_list)
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

Observation space: Box(-5000.0, 5000.0, (50, 1), float32)
Action space: Discrete(5)


In [15]:
# Create an agent
agent = QNAgent(env)

In [16]:
# Load model
if load_model:
    agent.load(path_models + name_model_weights + f'_{epoch_to_load}')
    train_statistics = pd.read_csv(path_stats + 'train.csv')
    test_statistics = pd.read_csv(path_stats + 'test.csv')
    init_ep = epoch_to_load
else:
    init_ep = 0
    train_statistics = pd.DataFrame()
    test_statistics = pd.DataFrame()

In [18]:
worth = []
for ep in range(init_ep, num_episodes):
    get_performance(env, agent, train_data=True, training=True, batch_size=batch_size)
    if (ep % interval) == 0:
        env.render(ep)
    worth.append(env.net_worth)
    if (ep % interval == 0) and not((load_model==True) and (ep == epoch_to_load)):
        agent.model.save_weights(path_models + name_model_weights)

        overlap = overlap
        results_train = np.empty(shape=(0, 3))
        results_test = np.empty(shape=(0, 3))

        size_test = ((len(env.eval_data)-env.len_obs-env.len_window) // overlap)+1
        cagr_train, vol_train = get_performance(env, agent, train_data=True, training=False, batch_size=size_test)
        results_train = np.array([np.tile(ep, size_test), cagr_train, vol_train]).transpose()

        cagr_test, vol_test = get_performance(env, agent, train_data=False, training=False, overlap=overlap, batch_size=size_test)
        results_test = np.array([np.tile(ep, size_test), cagr_test, vol_test]).transpose()

        train_statistics = pd.concat([train_statistics, pd.DataFrame(results_train, columns=['epoch', 'cagr','volatility'])])
        train_statistics.to_csv(path_stats+'train.csv', index=False)
        test_statistics = pd.concat([test_statistics, pd.DataFrame(results_test, columns=['epoch', 'cagr','volatility'])])
        test_statistics.to_csv(path_stats+'test.csv', index=False)


ep 0 
Profit: 5.707104969400001%
Std: 0.1790126773940898

ep 100 
Profit: 3.5239260565000015%
Std: 0.15806507139701118

ep 200 
Profit: 4.9188194998000005%
Std: 0.1831786884554391


KeyboardInterrupt: 

        plot_stocks_trading_performance(results_train, path_imgs + f'train_cagr_vol_ep_{ep}',
                                        color='royalblue', acc_title='Train')
        plot_stocks_trading_performance(results_test, path_imgs + f'test_cagr_vol_ep_{ep}',
                                        color='firebrick', acc_title='Test')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.mean(worth,axis=-1))